In [1]:
import torch
from transformer_lens import HookedTransformer
from transformer_lens import utils

from functools import partial

from sae_lens import SparseAutoencoder
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

torch.set_grad_enabled(False)


In [2]:
model: HookedTransformer = HookedTransformer.from_pretrained('gpt2-small', device='cpu')


Loaded pretrained model gpt2-small into HookedTransformer


In [3]:
# layer = 1
# prompt_pos = "Yes, I talk about wedding constantly"
# prompt_neg = "I do not talk about wedding constantly"
# prompt_pos = "Love "
# prompt_neg = "Hate"
prompt_pos = "Anger"
prompt_neg = "Calm"

In [4]:
logits, pos_cache = model.run_with_cache(prompt_pos)
# h_p = pos_cache["resid_pre", layer]

logits, neg_cache = model.run_with_cache(prompt_neg)
# h_n = neg_cache["resid_pre", layer]

# print(h_p.shape, h_n.shape)
# steering = h_p - h_n
# steering.shape

In [5]:
# c = 5
def residual_stream_patching_hook(
    resid,
    hook,
    c
):
    h_p = pos_cache[hook.name]
    h_n = neg_cache[hook.name]
    steering = h_p - h_n

    len_steering = steering.shape[1]
    
    # resid shape is (batch, pos, d_model)
    # print(resid.shape, steering.shape)
    resid[:, :len_steering, :] =  resid[:, :len_steering, :] + c * steering

    return resid

In [6]:
text = "I think you're"
n_samples = 7
hook_name = utils.get_act_name("resid_pre", 7)

with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook, c=5))]):
    # model('testing testing asdf;lkjasdf;l k')
    for i in range(n_samples):
        print("============")
        output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=20, verbose=False)
        print(output)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


I think you're correct... This was a problem with the final one. So, Unlimitedly was vandalized in Op
I think you're forgetting something. The chain restriction was meant to prevent illegal traffic from appearing in the area or at other
I think you're aware this post is getting on. The tech community at large continues to obsess over the cause of open
I think you're capable of adapting and embracing, as well as adapting yourself to changing situations and gentrification in New York


KeyboardInterrupt: 

In [ ]:
love_words = ["love", "like", "adore", "enjoy", "appreciate", "cherish", "admire", "care", "fancy", "favor", "prefer"]
hate_words = ["hate", "dislike", "detest", "abhor", "despise", "scorn", "loathe", "despise", "fuck you"]

def compute_metric(positive_words, layer, n_samples, factor):
    count = 0
    hook_name = utils.get_act_name("resid_pre", layer)

    with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook, c=factor))]):
        for i in range(n_samples):
            output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=25, verbose=False)
            for word in positive_words:
                if word in output.lower():
                    count += 1
                    break
        
    return count/n_samples

In [ ]:
for l in range(model.cfg.n_layers):
    n_samples = 10
    score = compute_metric(hate_words, l, n_samples, factor=10)
    print(f"layer: {l}, socre: {score}")

layer: 0, socre: 0.0
layer: 1, socre: 0.0
layer: 2, socre: 0.0
layer: 3, socre: 0.0
layer: 4, socre: 0.0
layer: 5, socre: 0.0
layer: 6, socre: 0.0
layer: 7, socre: 0.0
layer: 8, socre: 0.0
layer: 9, socre: 0.0
layer: 10, socre: 0.0
layer: 11, socre: 0.0


In [7]:
layer = 7 # pick a layer you want.

sparse_autoencoder = SparseAutoencoder.from_pretrained(
    "gpt2-small-res-jb", f"blocks.{layer}.hook_resid_pre"
)

AttributeError: type object 'SparseAutoencoder' has no attribute 'from_pretrained'

In [8]:
hook_point = f"blocks.{layer}.hook_resid_pre"

saes, sparsities = get_gpt2_res_jb_saes(hook_point)

print(saes.keys())
sae = saes[hook_point]

  0%|          | 0/1 [00:00<?, ?it/s]

blocks.7.hook_resid_pre/cfg.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

(…)7.hook_resid_pre/sae_weights.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

(…)ks.7.hook_resid_pre/sparsity.safetensors:   0%|          | 0.00/98.4k [00:00<?, ?B/s]

100%|██████████| 1/1 [00:17<00:00, 17.13s/it]

dict_keys(['blocks.7.hook_resid_pre'])


In [9]:
### cache angner and then pass through sae, find anger feature, add anger feature during the forward pass.

hook_name = utils.get_act_name("resid_pre", layer)



In [53]:

logits, cache = model.run_with_cache("Anger")
anger_hidden_state = cache[hook_name][0, -1, :].unsqueeze(0)

feature_acts = sae(anger_hidden_state).feature_acts[0]
top_values, top_ids = torch.topk(feature_acts, 10)
print(top_values)
print(top_ids)

tensor([18.4649, 16.4535, 12.0989, 11.0684,  7.7472,  7.2738,  5.0492,  4.7868,
         4.7161,  4.6675])
tensor([16077, 21456,  6857, 23357, 19453, 14237, 12147, 21901, 20881,  9111])


In [62]:
logits, cache = model.run_with_cache("Sad")
anger_hidden_state = cache[hook_name][0, -1, :].unsqueeze(0)

feature_acts = sae(anger_hidden_state).feature_acts[0]
top_values, top_ids = torch.topk(feature_acts, 10)
print(top_values)
print(top_ids)

tensor([39.3675, 16.9168, 15.9074,  5.2266,  5.0246,  3.7320,  3.0697,  3.0380,
         2.8858,  2.3728])
tensor([ 2045, 21248, 20750, 16744, 20385,   667,  4879,  9875, 16910,  8835])


In [14]:
print((feature_acts != 0).sum())

tensor(78)


In [15]:
l1_contribution = top_values[0]/feature_acts.sum()
l1_contribution

tensor(0.1017)

In [63]:
target_feature = top_ids[0]

# only top feature
steering = sae.W_dec[top_ids[0]] * top_values[0]   #sae.W_dec[top_ids[:5]].sum(dim=0) # shape: [d_model]

# top 5 features
# steering = torch.stack([sae.W_dec[top_ids[i]] * top_values[i] for i in range(5)]).sum(dim=0)

steering.shape

torch.Size([768])

In [65]:
# c = 5
def residual_stream_patching_hook_sae(
    resid,
    hook,
    c,
    pos
):    
    # resid shape is (batch, pos, d_model)
    # print(resid.shape, steering.shape)
    if pos < resid.shape[1]:
        resid[:, pos, :] = resid[:, pos, :] + c * steering

    return resid


text = "I think you're"
n_samples = 7

with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook_sae, c=1))]):
    # model('testing testing asdf;lkjasdf;l k')
    for i in range(n_samples):
        print("============")
        output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=20, verbose=False)
        print(output)

In [69]:
love_words = ["love", "like", "adore", "enjoy", "appreciate", "cherish", "admire", "care", "fancy", "favor", "prefer"]
angry_words = ["angry", "mad", "pissed", "irritated", "annoyed", "frustrated", "enraged", "furious", "infuriated", "outraged", "resentful", "agitated", "aggravated", "bitter", "hate", "fuck"]

def compute_metric(related_words, n_samples, coef, pos, max_new_tokens=20):
    count = 0

    with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook_sae, c=coef, pos=pos))]):
        for i in range(n_samples):
            output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=max_new_tokens, verbose=False)
            for word in related_words:
                if word in output.lower():
                    count += 1
                    break
        
    return count/n_samples


n_samples = 2
max_new_tokens = 5
score_matrix = torch.zeros((10, 4))
for pos in range(10):
    for ci, c in enumerate([0.5, 1, 5, 10]):
        score = compute_metric(angry_words, n_samples, c, pos, max_new_tokens)
        score_matrix[pos, ci] = score

        print(f'pos: {pos}, c: {c}, socre: {score}')

pos: 0, c: 0.5, socre: 0.0
pos: 0, c: 1, socre: 0.0
